In [82]:
from tianshou.data import Batch, ReplayBuffer
import numpy as np

buffer = ReplayBuffer(size=5)

for i in range(5):
    buffer.add(Batch(obs=[-i,i,i+20], act=i, rew=i, done=(i==4), obs_next=[-(i+1), i+1, i+21], info={'future_length':(4-i)}))

achieved_goal_index = 1
desired_goal_index = 2
batch_size = 4
future_p = 0.8
transitions, indices = buffer.sample(batch_size)
# index of transitions to be replaced
replace_idx = np.random.choice(batch_size, int(batch_size*future_p))
replace_indices = indices[replace_idx]
# get future goal
future_goal_indices = replace_indices + (np.random.uniform(size=len(replace_idx)) * transitions[replace_idx].info.future_length).astype(int) 

In [46]:
obs = transitions.obs[replace_idx]
obs_future = buffer.obs[future_goal_indices]
transitions.obs[replace_idx] = obs_future
print(obs_future, transitions.obs[replace_idx])

[[ 1 21]
 [ 0 20]
 [ 4 24]
 [ 4 24]] [[ 1 21]
 [ 0 20]
 [ 4 24]
 [ 4 24]]


In [83]:
# replace goal and reward
obs_future = buffer.obs[future_goal_indices]
obs_old = transitions.obs[replace_idx]
obs_new = np.concatenate((obs_old[:, :desired_goal_index], obs_future[:, achieved_goal_index:desired_goal_index]), axis = 1)
print(obs_future, replace_idx)
transitions.obs[replace_idx] = obs_new
obs_new = transitions.obs[replace_idx]
for i, idx in enumerate(replace_idx):
    transitions.rew[idx] = (obs_new[i, achieved_goal_index:desired_goal_index] + obs_new[i, desired_goal_index:] + 100)

[[-3  3 23]
 [-2  2 22]
 [-3  3 23]] [1 1 2]
Batch(
    obs: array([[ 0,  0, 20],
                [-2,  2, 22],
                [-2,  2, 22],
                [-2,  2, 22]]),
    act: array([0, 2, 2, 2]),
    rew: array([0., 2., 2., 2.]),
    done: array([False, False, False, False]),
    obs_next: array([[-1,  1, 21],
                     [-3,  3, 23],
                     [-3,  3, 23],
                     [-3,  3, 23]]),
    info: Batch(
              future_length: array([4, 2, 2, 2]),
          ),
    policy: Batch(),
)
Batch(
    obs: array([[ 0,  0, 20],
                [-2,  2,  2],
                [-2,  2,  3],
                [-2,  2, 22]]),
    act: array([0, 2, 2, 2]),
    rew: array([0., 2., 2., 2.]),
    done: array([False, False, False, False]),
    obs_next: array([[-1,  1, 21],
                     [-3,  3, 23],
                     [-3,  3, 23],
                     [-3,  3, 23]]),
    info: Batch(
              future_length: array([4, 2, 2, 2]),
          ),
    poli

In [78]:
a = np.array([[-3 , 3],
 [-1  ,1],
 [-1 , 1]])
b = np.array([[3],
 [2],
 [3]])
np.concatenate((a,b),axis=1)

array([[-3,  3,  3],
       [-1,  1,  2],
       [-1,  1,  3]])